In [ ]:
import os
import os.path as op

import numpy as np
import matplotlib.pyplot as plt

# mne
import mne
from mne.io import read_raw_egi

# mypy
from mypy.chans import correct_egi_channel_names
from mypy.events import get_events_from_din
from mypy.viz import set_3d_axes_equal

%matplotlib

In [ ]:
base_dir = r'C:\Users\Ola\Dropbox\CAT N170\EEG'
file_name = r'CAT312 20131209 2056002.raw'

In [11]:
file_path = op.join(base_dir, file_name)
print(file_path)

# wczytujemy plik
raw = read_raw_egi(file_path, preload=True, verbose=False)
print('\n', raw)

# filtrowanie
raw.filter(0.5, None)

# wczytujemy montaż
montage = mne.channels.read_montage('GSN-HydroCel-65_1.0')
correct_egi_channel_names(raw)
raw.set_montage(montage)

# bierzemy wydarzenia z kanałów DIN
events = get_events_from_din(raw)

# teraz tworzymy słownik tylko z wydarzeniami, względem których będziemy epokować:
event_id = { "car/0": 36, "car/90": 40, "car/180": 44,
            "face/0": 52, "face/90": 56, "face/180": 60}

# epokujemy:
epochs = mne.Epochs(raw, events, tmin=-0.25, tmax=0.75,
                    event_id=event_id, preload=True)

# ustawiamy referencję do średniej - porównamy sobie co nam zmienia
# (o ref do średniej zrobię jeszcze jakiś tutorial / notebook)
epochs.set_eeg_reference()

# zwienczenie - plot, usuwamy teraz złe epoki
epochs.plot(n_epochs=6, n_channels=64, scalings={'eeg': 'auto'});

C:\Users\Ola\Dropbox\CAT N170\EEG\CAT312 20131209 2056002.raw

 <RawEGI  |  CAT312 20131209 2056002.raw, n_channels x n_times : 73 x 173724 (694.9 sec), ~96.9 MB, data loaded>
393 matching events found
0 projection items activated
Loading data for 393 events and 251 original time points ...
0 bad epochs dropped
Adding average EEG reference projection.
1 projection items deactivated
Dropped 7 epochs
Channels marked as bad: ['E19']


In [12]:
# po usunięciu złych epok - zapisujemy ocalałe epoki z rozszerzeniem `-epo.fif`
epochs.save(op.join(base_dir, file_name.replace('.raw', '-epo.fif')))

## ICA

In [13]:
from mne.preprocessing import ICA

ica = ICA(method='extended-infomax', n_components=0.99)
ica.fit(epochs)

Fitting ICA to data using 64 channels. 
Please be patient, this may take some time
Inferring max_pca_components from picks.
Selection by explained variance: 42 components
computing Extended Infomax ICA


<ICA  |  epochs decomposition, fit (extended-infomax): 96886 samples, 42 components, channels used: "eeg">

In [18]:
# otwieramy podgląd pierwszych 25 komponentów oraz okno z ich sygnałem
ica.plot_components(picks=list(range(0, 25)), inst=epochs);
ica.plot_sources()

In [22]:
# gdy chemy zobaczyć z rozmyciem:
ica.plot_properties(picks=[0, 3, 4, 5], inst=epochs, image_args=dict(sigma=2.5));

In [20]:
# upewniamy się, że zostały zaznaczone złe kompo
ica.exclude

[9, 8, 3, 4, 5, 6]

In [21]:
# i zapisujemy z rozszerzeniem `-ica.fif`
ica.save(op.join(base_dir, file_name.replace('.raw', '-ica.fif')))

Writing ica solution to C:\Users\Ola\Dropbox\CAT N170\EEG\ica\CAT312 20131209 2056002-ica.fif...


<ICA  |  epochs decomposition, fit (extended-infomax): 96886 samples, 42 components, channels used: "eeg", 6 sources marked for exclusion>